In [1]:
import pandas as pd
import ast
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sentence_transformers import util

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
credits = pd.read_csv("/content/drive/MyDrive/tmdb_5000_credits.csv")
movies = pd.read_csv("/content/drive/MyDrive/tmdb_5000_movies.csv")

In [4]:
movies = movies.merge(credits, on="title")
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [6]:
movies = movies[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew", "popularity", "tagline"]]

In [7]:
movies.dropna(inplace=True)

In [8]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i["name"])
    return l

In [9]:
movies.genres = movies.genres.apply(convert)

In [10]:
movies.keywords = movies.keywords.apply(convert)

In [11]:
def convert2(obj):
    l = []
    count = 0
    for i in ast.literal_eval(obj):
        if count != 10:
            l.append(i['name'])
            count += 1
        else:
            break
    return l

In [12]:
movies.cast = movies.cast.apply(convert2)

In [13]:
def convert3(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i["job"] == "Director":
            l.append(i["name"])
        else:
            continue
    return l
movies.crew = movies.crew.apply(convert3)
movies.overview = movies.overview.apply(lambda x:x.split())

In [14]:
movies.genres = movies.genres.apply(lambda x:[i.replace(" ", "")*2 for i in x])
movies.keywords = movies.keywords.apply(lambda x:[i.replace(" ", "") for i in x])
movies.cast = movies.cast.apply(lambda x:[i.replace(" ", "") for i in x])
movies.crew = movies.crew.apply(lambda x:[i.replace(" ", "") for i in x])

In [15]:
movies.genres

,genres
0,"[ActionAction, AdventureAdventure, FantasyFant..."
1,"[AdventureAdventure, FantasyFantasy, ActionAct..."
2,"[ActionAction, AdventureAdventure, CrimeCrime]"
3,"[ActionAction, CrimeCrime, DramaDrama, Thrille..."
4,"[ActionAction, AdventureAdventure, ScienceFict..."
...,...
4801,[DramaDrama]
4802,"[ScienceFictionScienceFiction, DramaDrama, Thr..."
4804,"[ActionAction, CrimeCrime, ThrillerThriller]"
4805,"[ComedyComedy, RomanceRomance]"


In [18]:
movies['popularity'] = movies['popularity'].apply(lambda x: [str(x)])
movies['tagline'] = movies['tagline'].apply(lambda x: [] if pd.isna(x) else x.split())
movies["tags"] = (
    movies['overview']
    + movies['genres']
    + movies['keywords']
    + movies['cast']
    + movies['crew']
    + movies['popularity']
    + movies['tagline']
)

df = movies[['movie_id', 'title', 'tags']]

df.tags = df.tags.apply(lambda x: " ".join(x))

/tmp/ipython-input-3403174323.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.tags = df.tags.apply(lambda x: " ".join(x))


In [19]:
df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [20]:
lemmatizer = WordNetLemmatizer()

In [21]:
def stem(text):
    y = [lemmatizer.lemmatize(i) for i in text.split()]
    return " ".join(y)
df.tags.apply(stem)

,tags
0,"In the 22nd century, a paraplegic Marine is di..."
1,"Captain Barbossa, long believed to be dead, ha..."
2,A cryptic message from Bond’s past sends him o...
3,Following the death of District Attorney Harve...
4,"John Carter is a war-weary, former military ca..."
...,...
4801,A young woman in L.A. is having a bad day: she...
4802,Friends/fledgling entrepreneur invent a device...
4804,El Mariachi just want to play his guitar and c...
4805,A newlywed couple's honeymoon is upended by th...


In [75]:
tfidf = TfidfVectorizer(max_features=3000, stop_words="english")
vectors = tfidf.fit_transform(df.tags).toarray()
similarity = cosine_similarity(vectors)

In [76]:
def recommand(movie):
    movie_index = df[df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:21]
    for i in movies_list:
        print(df.iloc[i[0]].title)

In [87]:
recommand("Ant-Man")

The November Man
Bridge of Spies
Topaz
Mission: Impossible
Spy Game
Central Intelligence
The East
Patriot Games
Safe House
Clear and Present Danger
Abduction
Charlie Wilson's War
The Bourne Supremacy
This Means War
Fair Game
The Sum of All Fears
Taken 2
Bad Company
The Man from U.N.C.L.E.
Zero Dark Thirty
